<a href="https://colab.research.google.com/github/Ab2207/Academic-Projects-Data-Analytics/blob/main/Masters%20Thesis%20Experiment%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np 
import string 
string.punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
import re
import unicodedata

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/BLM_Tweets_1M.csv")

In [ ]:
def tweets_cleaning(text):
    lowercase = text.lower()
    punc_removal = [char for char in lowercase if char not in string.punctuation]
    punc_removal_joined = ''.join(punc_removal)
    url_removal = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', punc_removal_joined, flags=re.MULTILINE)
    emoji_removal = url_removal.encode('ascii', 'ignore').decode('ascii')
    stopwords_removal = [word for word in emoji_removal.split() if word not in stopwords.words('english')]
    return stopwords_removal

In [ ]:
data['cleaned_text'] = data['text'].apply(tweets_cleaning).astype(str)

In [ ]:
from textblob import TextBlob
data['polarity'] = data['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
conditionList = [
                 data['polarity'] < 0,
                 data['polarity'] >= 0]
choiceList = ['0', '1']
data['target'] = np.select(conditionList, choiceList, default='no_label')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(data['cleaned_text'].values)

word_index = tokenizer.word_index #word tokenizing

In [ ]:
vocab_size = len(word_index) + 1
print(vocab_size)

924810


In [ ]:
train_sentence_tokenizing = tokenizer.texts_to_sequences(data['cleaned_text'].values)
X = pad_sequences(train_sentence_tokenizing, padding = "post", truncating = "post", maxlen= 26)
print(X)

[[    2 45225 70438 ...     0     0     0]
 [ 7817  1520     2 ...     0     0     0]
 [  124  5478 91399 ...     0     0     0]
 ...
 [30034     2    56 ...     0     0     0]
 [    6     7  1245 ...     0     0     0]
 [ 1823  1238    29 ...     0     0     0]]


In [ ]:
embeddings_index = dict()
f = open('/content/gdrive/My Drive/glove.twitter.27B.200d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [ ]:
embedding_matrix = np.zeros((vocab_size, 200))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
y = pd.get_dummies(data['target']).values

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Activation, Input, Flatten, Bidirectional
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1234, test_size = 0.20, stratify = y)

In [ ]:
embed_dim = 200
lstm_out = 128

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1], weights=[embedding_matrix],trainable=False))
model.add(LSTM(lstm_out, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(lstm_out))
model.add(Dropout(0.3))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 200)           184962000 
_________________________________________________________________
lstm (LSTM)                  (None, 26, 128)           168448    
_________________________________________________________________
dropout (Dropout)            (None, 26, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
_________________________________________________________________
activation (Activation)      (None, 2)                 0

In [ ]:
batch_size = 320
model.fit(X_train, y_train, epochs = 5, verbose = 1, validation_data=(X_test, y_test), batch_size=batch_size)

Epoch 1/5
2508/2508 [==============================] - 31s 12ms/step - loss: 0.4817 - accuracy: 0.8141 - val_loss: 0.4800 - val_accuracy: 0.8144
Epoch 2/5
2508/2508 [==============================] - 31s 12ms/step - loss: 0.4806 - accuracy: 0.8144 - val_loss: 0.4803 - val_accuracy: 0.8144
Epoch 3/5
2508/2508 [==============================] - 31s 12ms/step - loss: 0.4804 - accuracy: 0.8144 - val_loss: 0.4799 - val_accuracy: 0.8144
Epoch 4/5
2508/2508 [==============================] - 31s 12ms/step - loss: 0.4802 - accuracy: 0.8144 - val_loss: 0.4799 - val_accuracy: 0.8144
Epoch 5/5
2508/2508 [==============================] - 31s 13ms/step - loss: 0.4801 - accuracy: 0.8144 - val_loss: 0.4799 - val_accuracy: 0.8144


In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 81.435239
